## Intro to the Data

In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect("nominations.db")

In [3]:
schema_query = "PRAGMA table_info(nominations);"
schema = conn.execute(schema_query).fetchall()

In [4]:
first_10_query = "SELECT * FROM nominations LIMIT 10;"
first_ten = conn.execute(first_10_query).fetchall()

In [5]:
for row in schema:
    print(row)

(0, 'Year', 'INTEGER', 0, None, 0)
(1, 'Category', 'TEXT', 0, None, 0)
(2, 'Nominee', 'TEXT', 0, None, 0)
(3, 'Won', 'INTEGER', 0, None, 0)
(4, 'Movie', 'TEXT', 0, None, 0)
(5, 'Character', 'TEXT', 0, None, 0)


In [6]:
for row in first_ten:
    print(row)

(2010, 'Actor -- Leading Role', 'Javier Bardem', 0, 'Biutiful', 'Uxbal')
(2010, 'Actor -- Leading Role', 'Jeff Bridges', 0, 'True Grit', 'Rooster Cogburn')
(2010, 'Actor -- Leading Role', 'Jesse Eisenberg', 0, 'The Social Network', 'Mark Zuckerberg')
(2010, 'Actor -- Leading Role', 'Colin Firth', 1, "The King's Speech", 'King George VI')
(2010, 'Actor -- Leading Role', 'James Franco', 0, '127 Hours', 'Aron Ralston')
(2010, 'Actor -- Supporting Role', 'Christian Bale', 1, 'The Fighter', 'Dicky Eklund')
(2010, 'Actor -- Supporting Role', 'John Hawkes', 0, "Winter's Bone", 'Teardrop')
(2010, 'Actor -- Supporting Role', 'Jeremy Renner', 0, 'The Town', 'James Coughlin')
(2010, 'Actor -- Supporting Role', 'Mark Ruffalo', 0, 'The Kids Are All Right', 'Paul')
(2010, 'Actor -- Supporting Role', 'Geoffrey Rush', 0, "The King's Speech", 'Lionel Logue')


## Creating The Ceremonies Table

In [7]:
create_query = '''
create table ceremonies(
id integer primary key,
Year integer,
Host text
);
'''
conn.execute(create_query)

OperationalError: table ceremonies already exists

In [8]:
year_hosts = [(2010, "Steve Martin"),
              (2009, "Hugh Jackman"),
              (2008, "Jon Stewart"),
              (2007, "Ellen DeGeneres"),
              (2006, "Jon Stewart"),
              (2005, "Chris Rock"),
              (2004, "Billy Crystal"),
              (2003, "Steve Martin"),
              (2002, "Whoopi Goldberg"),
              (2001, "Steve Martin"),
              (2000, "Billy Crystal")
             ]

In [9]:
insert_query = "insert into ceremonies (Year, Host) Values (?,?);"
conn.executemany(insert_query, year_hosts)

In [10]:
query_ceremonies = "select * from ceremonies limit 10;"
first_10 = conn.execute(query_ceremonies).fetchall()
for row in first_10:
    print(row)

(1, 2010, 'Steve Martin')
(2, 2009, 'Hugh Jackman')
(3, 2008, 'Jon Stewart')
(4, 2007, 'Ellen DeGeneres')
(5, 2006, 'Jon Stewart')
(6, 2005, 'Chris Rock')
(7, 2004, 'Billy Crystal')
(8, 2003, 'Steve Martin')
(9, 2002, 'Whoopi Goldberg')
(10, 2001, 'Steve Martin')


In [11]:
pragma_query2 = "pragma table_info(ceremonies);"
pragma2 = conn.execute(pragma_query2).fetchall()
for row in pragma2:
    print(row)

(0, 'id', 'integer', 0, None, 1)
(1, 'Year', 'integer', 0, None, 0)
(2, 'Host', 'text', 0, None, 0)


## Foreign Key Constraints

In [12]:
fkeys_query = "pragma foreign_keys = ON;"
conn.execute(fkeys_query)

## Setting Up One-To-Many

In [21]:
nom_two_create_new1 = '''
create table nominations_two_new1(
id integer primary key,
category text,
nominee text,
movie text,
character, text,
won integer,
ceremony_id integer,
foreign key (ceremony_id) references ceremonies(id));
'''

In [23]:
conn.execute(nom_two_create_new1)

In [24]:
conn.execute("pragma table_info(nominations_two_new1)").fetchall()

[(0, 'id', 'integer', 0, None, 1),
 (1, 'category', 'text', 0, None, 0),
 (2, 'nominee', 'text', 0, None, 0),
 (3, 'movie', 'text', 0, None, 0),
 (4, 'character', '', 0, None, 0),
 (5, 'text', '', 0, None, 0),
 (6, 'won', 'integer', 0, None, 0),
 (7, 'ceremony_id', 'integer', 0, None, 0)]

In [25]:
nom_query = '''
select ceremonies.id as ceremony_id, nominations.category as category, 
nominations.nominee as nominee, nominations.movie as movie, 
nominations.character as character, nominations.won as won
from nominations
inner join ceremonies 
on nominations.year == ceremonies.year
;
'''

In [26]:
joined_nominations = conn.execute(nom_query).fetchall()
for row in joined_nominations:
    print(row)

(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0)
(12, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0)
(23, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', 0)
(1, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', 0)
(12, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', 0)
(23, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', 0)
(1, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', 0)
(12, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', 0)
(23, 'Actor -- Leading Role', 'Jesse Eisenberg', 'The Social Network', 'Mark Zuckerberg', 0)
(1, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', 1)
(12, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', 1)
(23, 'Actor -- Leading Role', 'Colin Firth', "The King's Speech", 'King George VI', 1)
(1, 'Act

In [27]:
insert_nominations_two = '''insert into nominations_two_new1 (ceremony_id, category, nominee, movie, character, won) 
values (?,?,?,?,?,?);
'''

In [28]:
conn.executemany(insert_nominations_two, joined_nominations)

In [31]:
print(conn.execute("select * from nominations_two_new1 limit 5;").fetchall())

[(1, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', None, 0, 1), (2, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', None, 0, 12), (3, 'Actor -- Leading Role', 'Javier Bardem', 'Biutiful', 'Uxbal', None, 0, 23), (4, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', None, 0, 1), (5, 'Actor -- Leading Role', 'Jeff Bridges', 'True Grit', 'Rooster Cogburn', None, 0, 12)]


## Deleting and Renaming Table

In [32]:
drop_query = "drop table nominations"
conn.execute(drop_query)

In [34]:
alter_query = "alter table [nominations_two_new1] rename to [nominations]"
conn.execute(alter_query)

## Creating a Join Table

In [35]:
movies_query = "create table movies(id integer primary key, movie text)"
conn.execute(movies_query)

In [36]:
actors_query = "create table actors (id integer primary key, actor text)"
conn.execute(actors_query)

In [38]:
movie_actors_query = "create table movie_actors (id integer primary key, movie_id integer references movies(id), actor_id integer references actors(id));"
conn.execute(movie_actors_query)

In [39]:
print(conn.execute("select * from movie_actors limit 5").fetchall())

[]


## Populating Movie & Actor Tables

In [41]:
movie_insert = "insert into movies (movie) select distinct movie from nominations;"
actor_insert = "insert into actors (actor) select distinct nominee from nominations;"

conn.execute(movie_insert)
conn.execute(actor_insert)

In [42]:
print(conn.execute("select * from movies limit 5").fetchall())
print(conn.execute("select * from actors limit 5").fetchall())

[(1, 'Biutiful'), (2, 'True Grit'), (3, 'The Social Network'), (4, "The King's Speech"), (5, '127 Hours')]
[(1, 'Javier Bardem'), (2, 'Jeff Bridges'), (3, 'Jesse Eisenberg'), (4, 'Colin Firth'), (5, 'James Franco')]


## Populating Join Table

In [43]:
pairs_query = "select movie,nominee from nominations;"
movie_actors_pairs = conn.execute(pairs_query).fetchall()

In [46]:
movies_actors_insert = '''
insert into movie_actors (movie_id, actor_id) 
values ((select id from movies where movie == ?), 
(select id from actors where actor == ?));
'''

In [47]:
conn.executemany(movies_actors_insert, movie_actors_pairs)

In [49]:
print(conn.execute("select * from movie_actors limit 5").fetchall())

[(1, 1, 1), (2, 1, 1), (3, 1, 1), (4, 2, 2), (5, 2, 2)]
